In [1]:
from os.path import join
import numpy as np
from numpy import array
import string 
import gensim
import time
import json
word2vec = gensim.models.Word2Vec

In [63]:
import ipynb.fs.full.conll_pipeline2 as train

SyntaxError: invalid syntax (conll_pipeline2.ipynb, line 426)

In [58]:
path = '/home/vishesh/TUM/Thesis/coref-json/trainfile1.json'
path_to_docs = '/home/vishesh/TUM/Thesis/coref-json/documents.json'

In [59]:
GLOVE_DIR = "/home/vishesh/TUM/Thesis/glove6B"

In [60]:
train_file = json.load(open(path))

In [61]:
docs_file = json.load(open(path_to_docs))

In [6]:
#train_file

In [7]:
model = gensim.models.KeyedVectors.load_word2vec_format(join(GLOVE_DIR, 'glove.6B.50d.w2vformat.txt'), binary=False)

In [8]:
def get_vector(word):
    table = str.maketrans({key: None for key in string.punctuation})
    word = word.lower()
    if len(word) > 1:
        word = word.translate(table)
    try:
        vec = model[word]
    except:
        vec = np.zeros((50, 1))
    return vec.reshape((50, 1))

In [9]:
get_vector('20').shape

(50, 1)

In [10]:
def get_average_vector(word_list):
    sum = np.zeros((50, 1))
    for i in range(0, len(word_list)):
        sum += get_vector(word_list[i])
    average_vector = sum/(i+1)
    return average_vector

In [11]:
def calculate_docs_average():
    doc_avg = []
    for d in docs_file:
        doc_avg.append(get_average_vector(docs_file[str(d)].split()))
    return doc_avg

In [56]:
def get_pair_features(feature_list):
    
    # distance features
    mention_dist = np.array(feature_list[4]['mention_dist_count']).reshape((10, 1))
    s_dist = np.array(feature_list[3]['sentence_dist_count']).reshape((10, 1))
    overlap = np.array(feature_list[5]['overlap']).reshape((1, 1))
    
    # speaker feature
    speaker = np.array(feature_list[6]['speaker']).reshape((1, 1))
    
    # string matching features
    head_match = np.array(feature_list[7]['head_match']).reshape((1, 1))
    mention_exact_match = np.array(feature_list[8]['mention_exact_match']).reshape((1, 1))
    mention_partial_match = np.array(feature_list[9]['mention_partial_match']).reshape((1, 1))
    
    pair_features = np.concatenate((mention_dist, s_dist, overlap, speaker, head_match, \
                                   mention_exact_match, mention_partial_match))
    
    return pair_features

In [51]:
# p: previous, n: next, w: words, a: average, s: sentence
def get_mention_features(mention, doc_average):
    features = []
    #head_w = get_vector(mention['head_word'])
    first_w = get_vector(mention['first_word'])
    last_w = get_vector(mention['last_word'])
    mention_length = np.array(mention['mention_length']).reshape((1,1))
    mention_type = np.array(mention['mention_type']).reshape((4, 1))
    mention_position = np.array(mention['mention_position']).reshape((1, 1))
    if mention['contained'] == False:
        mention_contain = np.zeros((1, 1))
    else:
        mention_contain = np.ones((1, 1))
    if len(mention['pre_words']) > 0:
        mention_p_w1 = get_vector(mention['pre_words'][0])
    else:
        mention_p_w1 = np.zeros((50, 1))
    if len(mention['pre_words']) > 1:
        mention_p_w2 = get_vector(mention['pre_words'][1])
    else:
        mention_p_w2 = np.zeros((50, 1))
    if len(mention['next_words']) > 0:
        mention_n_w1 = get_vector(mention['next_words'][0])
    else:
        mention_n_w1 = np.zeros((50, 1))
    if len(mention['next_words']) > 1:
        mention_n_w2 = get_vector(mention['next_words'][1])
    else:
        mention_n_w2 = np.zeros((50, 1))
    if len(mention['pre_words']) > 0:
        mention_p_w_a = get_average_vector(mention['pre_words'])
    else:
        mention_p_w_a = np.zeros((50, 1))
    if len(mention['next_words']) > 0:
        mention_n_w_a = get_average_vector(mention['next_words'])
    else:
        mention_next_words_avg = np.zeros((50, 1))
    mention_s_a = get_average_vector(mention['mention_sentence'].split())
    doc_id = mention['id'].split('_')[0]
    doc_avg = doc_average[int(doc_id)]
    
    
    features = np.concatenate((first_w, last_w, mention_p_w1, mention_p_w2, mention_p_w_a, \
                               mention_n_w1, mention_n_w2, mention_n_w_a, mention_s_a, mention_length, \
                               mention_type, mention_position, mention_contain, doc_avg))
    return features

In [52]:
def make_feature_input():
    docs_avg = calculate_docs_average()
    input_feature_list = []
    i = 0
    for m in train_file:
        i += 1
        input_feature_vector = []
        mention_avg = get_average_vector(m[0]['mention'].split())
        antecedent_avg = get_average_vector(m[1]['mention'].split())
        mention_features = get_mention_features(m[0], docs_avg)
        antecedent_features = get_mention_features(m[1], docs_avg)
        pair_features = get_pair_features(m)
        
        input_feature_vector.append(antecedent_avg)
        input_feature_vector.append(antecedent_features)
        input_feature_vector.append(mention_avg)
        input_feature_vector.append(mention_features)
        input_feature_vector.append(pair_features)
        input_feature_list.append(input_feature_vector)
        
    return input_feature_list


In [53]:
def make_input_vector():
    feature_input = make_feature_input()
    len_f_input = len(feature_input)
    input_ = []
    for f_input in feature_input:
        con = np.concatenate((f_input[0], f_input[1], f_input[2], f_input[3], f_input[4]))
        input_.append(con)
        del con
    return input_

In [54]:
def make_output_vector():
    output = []
    len_mentions = len(train_file)
    for m in train_file:
        output.append(m[2]['coref'])
    output = np.array(output).reshape((len_mentions, 1))
    return output    

In [62]:
%%time
input_ = make_input_vector()

CPU times: user 3.68 s, sys: 0 ns, total: 3.68 s
Wall time: 3.68 s


In [25]:
type(input_)

list

In [34]:
output_ = make_output_vector()

In [35]:
np.save('/home/vishesh/TUM/Thesis/coref-json/input_', input_, allow_pickle=True, fix_imports=True)

In [36]:
np.save('/home/vishesh/TUM/Thesis/coref-json/output_', output_, allow_pickle=True, fix_imports=True)